# Python Finance & Stocks

In [1]:
import datetime as dt 
import matplotlib.pyplot as plt 
from matplotlib import style #style your graphs
import pandas as pd 
import pandas_datareader.data as web #used to get stock data
from pandas.testing import assert_frame_equal #fixes warning message
import matplotlib.dates as mdates
import bs4 as bs
import pickle
import requests
import pyodbc
from pandas.tseries.holiday import USFederalHolidayCalendar
from pandas.tseries.offsets import CustomBusinessDay
from pandas.tseries.offsets import Day, Easter
from pandas.tseries.holiday import AbstractHolidayCalendar,nearest_workday,Holiday
from dateutil.relativedelta import FR, MO, SA, SU, TH, TU, WE
import holidays 

C:\Users\Ryan Thelen\Anaconda3\lib\site-packages\pandas_datareader\compat\__init__.py:7: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  from pandas.util.testing import assert_frame_equal


In [2]:
USMemorialDay = Holiday(
    "Memorial Day", month=5, day=31, offset=pd.DateOffset(weekday=MO(-1))
)
USLaborDay = Holiday("Labor Day", month=9, day=1, offset=pd.DateOffset(weekday=MO(1)))
USColumbusDay = Holiday(
    "Columbus Day", month=10, day=1, offset=pd.DateOffset(weekday=MO(2))
)
USThanksgivingDay = Holiday(
    "Thanksgiving", month=11, day=1, offset=pd.DateOffset(weekday=TH(4))
)
USMartinLutherKingJr = Holiday(
    "Martin Luther King Jr. Day",
    start_date=dt.datetime(1986, 1, 1),
    month=1,
    day=1,
    offset=pd.DateOffset(weekday=MO(3)),
)
USPresidentsDay = Holiday(
    "Presidents Day", month=2, day=1, offset=pd.DateOffset(weekday=MO(3))
)
GoodFriday = Holiday("Good Friday", month=1, day=1, offset=[Easter(), Day(-2)])

EasterMonday = Holiday("Easter Monday", month=1, day=1, offset=[Easter(), Day(1)])

In [3]:
class NYSEHolidayCalendar(AbstractHolidayCalendar):
    """
    US Federal Government Holiday Calendar based on rules specified by:
    https://www.opm.gov/policy-data-oversight/
       snow-dismissal-procedures/federal-holidays/
    """

    rules = [
        Holiday("New Years Day", month=1, day=1, observance=nearest_workday),
        USMartinLutherKingJr,
        USPresidentsDay,
        USMemorialDay,
        Holiday("July 4th", month=7, day=4, observance=nearest_workday),
        USLaborDay,
        USColumbusDay,
        Holiday("Good Friday", month=1, day=1, offset=[Easter(), Day(-2)]),
        Holiday("Veterans Day", month=11, day=11, observance=nearest_workday),
        USThanksgivingDay,
        Holiday("Christmas", month=12, day=25, observance=nearest_workday),
    ]

In [4]:
calendar=NYSEHolidayCalendar()

In [5]:
nyse_holidays=calendar.holidays(start=dt.date.today()-pd.DateOffset(years=2),end=dt.date.today())

In [6]:
nyse_holidays

DatetimeIndex(['2018-05-28', '2018-07-04', '2018-09-03', '2018-10-08',
               '2018-11-12', '2018-11-22', '2018-12-25', '2019-01-01',
               '2019-01-21', '2019-02-18', '2019-04-19', '2019-05-27',
               '2019-07-04', '2019-09-02', '2019-10-14', '2019-11-11',
               '2019-11-28', '2019-12-25', '2020-01-01', '2020-01-20',
               '2020-02-17', '2020-04-10'],
              dtype='datetime64[ns]', freq=None)

In [7]:
def is_holiday(date):
    return date in nyse_holidays
    

In [8]:
def is_weekend(date):
    return date.isoweekday()==6 or date.isoweekday()==7
        

In [9]:
def previous_workday_holiday(date):
    """
    returns previous weekday used for observances
    """
    #date -= timedelta(days=1)
    while date.isoweekday() in (6,7) or is_holiday(date):
        # Mon-Fri are 0-4
        date -= dt.timedelta(days=1)
    return date

previous_workday_holiday(dt.datetime(2019,4,20))

datetime.datetime(2019, 4, 18, 0, 0)

In [10]:
def weekend_to_workday(date):
    """
    returns previous weekday used for observances
    """
    while not date.isoweekday() <6:
        # Mon-Fri are 0-4
        date -= timedelta(days=1)
    return date

## Get Index Symbols

In [11]:
def get_sp500_symbols():
    resp=requests.get('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
    soup=bs.BeautifulSoup(resp.text,"lxml")
    table=soup.find('table',{'class':'wikitable sortable'})
    tickers=[]
    for row in table.findAll('tr')[1:]:
        ticker=row.findAll('td')[0].text
        tickers.append(ticker.replace('\n',''))
    with open("sp500symbols.pickle","wb") as f:
        pickle.dump(tickers,f)
        #print(tickers)
    return tickers

In [12]:
def get_nas100_symbols():
    resp=requests.get('https://en.wikipedia.org/wiki/NASDAQ-100')
    soup=bs.BeautifulSoup(resp.text,"lxml")
    table=soup.find('table',{'class':'wikitable sortable','id':'constituents'})
    tickers=[]
    for row in table.findAll('tr')[1:]:
        ticker=row.findAll('td')[1].text
        tickers.append(ticker.replace('\n',''))
    with open("nas100symbols.pickle","wb") as f:
        pickle.dump(tickers,f)
        #print(tickers)
    return tickers

## Get Index Company Names

In [58]:
def get_sp500_names():
    resp=requests.get('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
    soup=bs.BeautifulSoup(resp.text,"lxml")
    table=soup.find('table',{'class':'wikitable sortable'})
    company_names=[]
    for row in table.findAll('tr')[1:]:
        company=row.findAll('td')[1].text
        company_names.append(company.replace('\n',''))
    with open("sp500names.pickle","wb") as f:
        pickle.dump(company_names,f)
        #print(tickers)
    return company_names

In [59]:
def get_nas100_names():
    resp=requests.get('https://en.wikipedia.org/wiki/NASDAQ-100')
    soup=bs.BeautifulSoup(resp.text,"lxml")
    table=soup.find('table',{'class':'wikitable sortable','id':'constituents'})
    company_names=[]
    for row in table.findAll('tr')[1:]:
        company=row.findAll('td')[0].text
        company_names.append(company.replace('\n',''))
    with open("nas100names.pickle","wb") as f:
        pickle.dump(company_names,f)
        #print(tickers)
    return company_names

In [60]:
sp500_symbols=get_sp500_symbols() #assign symbols to list variable
sp500_company_name=get_sp500_names() #assign company names to list variable

In [61]:
nas100_symbols=get_nas100_symbols()
nas100_company_name=get_nas100_names()

## Create Table of S&P 500 Companies and Symbols

In [63]:
sp500=pd.DataFrame({'Symbols':sp500_symbols,'Company Name':sp500_company_name})

In [64]:
nas100=pd.DataFrame({'Symbols':nas100_symbols,'Company Name':nas100_company_name})

In [66]:
sp500.Symbols[sp500['Symbols']=='BRK.B']='BRK-B'
sp500.Symbols[sp500['Symbols']=='BF.B']='BF-B'
sp500=sp500.set_index('Symbols')
nas100=nas100.set_index('Symbols')

In [67]:
style.use('ggplot') #style of graphs

## Get Current Date and One Year Back

In [21]:
def get_last_stock_day():
    return previous_workday_holiday(dt.date.today())
def get_start_stock_day():
    return previous_workday_holiday(dt.date.today()-pd.DateOffset(months=13))

end=get_last_stock_day()
start=get_start_stock_day()

## Get S&P 500 historical market data

In [22]:
df=web.DataReader(list(sp500.index),'yahoo',start,end)

In [23]:
latest_day=df['Adj Close'].iloc[-1].name
latest_day=dt.date.today()


def get_month(latest_day,months):
    return previous_workday_holiday(latest_day-pd.DateOffset(months=months))

def get_weeks(latest_day,days):
    return previous_workday_holiday(latest_day-pd.DateOffset(days=days))
        
def get_years(latest_day,years):
    return previous_workday_holiday(latest_day-pd.DateOffset(years=years))

def get_date(latest_day,days):
    return previous_workday_holiday(latest_day-pd.DateOffset(days=days))

    
previous_month=get_month(latest_day,1)
previous_3_month=get_month(latest_day,3)
previous_year=get_years(latest_day,1)
previous_day=get_date(latest_day,1)
latest_day=get_date(latest_day,0)
previous_week=get_date(latest_day,7)

## S&P 500 Companies Percent Change

In [70]:
percent_change_prev_day=pd.Series(((df['Adj Close'].loc[latest_day]-df['Adj Close'].loc[previous_day])/df['Adj Close'].loc[previous_day])*100,name='Day Percent Change')
percent_change_abs_prev_day=pd.Series(abs((df['Adj Close'].iloc[-1,:]-df['Adj Close'].loc[previous_day])/df['Adj Close'].loc[previous_day])*100,name='Day Absolute Percent Change')

percent_change_prev_week=pd.Series(((df['Adj Close'].loc[latest_day]-df['Adj Close'].loc[previous_week])/df['Adj Close'].loc[previous_week])*100,name='Week Percent Change')
percent_change_abs_prev_week=pd.Series(abs((df['Adj Close'].loc[latest_day]-df['Adj Close'].loc[previous_week])/df['Adj Close'].loc[previous_week])*100,name='Week Absolute Percent Change')

percent_change_prev_month=pd.Series(((df['Adj Close'].loc[latest_day]-df['Open'].loc[previous_month])/df['Adj Close'].loc[previous_month])*100,name='Month Percent Change')
percent_change_abs_prev_month=pd.Series(abs((df['Adj Close'].iloc[-1,:]-df['Adj Close'].loc[previous_month])/df['Adj Close'].loc[previous_month])*100,name='Month Absolute Percent Change')

percent_change_prev_3_month=pd.Series(((df['Adj Close'].loc[latest_day]-df['Adj Close'].loc[previous_3_month])/df['Adj Close'].loc[previous_3_month])*100,name='3 Month Percent Change')
percent_change_abs_prev_3_month=pd.Series(abs((df['Adj Close'].iloc[-1,:]-df['Adj Close'].loc[previous_3_month])/df['Adj Close'].loc[previous_3_month])*100,name='3 Month Absolute Percent Change')

percent_change_prev_year=pd.Series(((df['Adj Close'].loc[latest_day]-df['Adj Close'].loc[previous_year])/df['Adj Close'].loc[previous_year])*100,name='Year Percent Change')
percent_change_abs_prev_year=pd.Series(abs((df['Adj Close'].iloc[-1,:]-df['Adj Close'].loc[previous_year])/df['Adj Close'].loc[previous_year])*100,name='Year Absolute Percent Change')

percent_change=pd.concat([percent_change_prev_day,percent_change_abs_prev_day,percent_change_prev_week,
                          percent_change_abs_prev_week,percent_change_prev_month,percent_change_abs_prev_month,
                          percent_change_prev_3_month,percent_change_abs_prev_3_month,
                          percent_change_prev_year,percent_change_abs_prev_year],axis=1)


percent_change=percent_change.join(sp500)

percent_change=percent_change[percent_change.columns[0:-1].insert(0,percent_change.columns[-1])]

In [71]:
percent_change.sort_values('Year Percent Change',ascending=False).reset_index().style.background_gradient(cmap='Blues')

## S&P 500 Top 10 Best Performing Stocks Day

In [26]:
percent_change.reset_index()[['Symbols','Company Name','Day Percent Change']].sort_values('Day Percent Change',ascending=False).reset_index(drop=True).head(10).style.background_gradient(cmap='Greens')

,Symbols,Company Name,Day Percent Change
0,HOG,HOG,15.205909
1,ADS,ADS,15.003622
2,NCLH,NCLH,14.423082
3,ALK,ALK,13.393176
4,AAL,AAL,12.375247
5,DHI,DHI,11.446211
6,CCL,CCL,11.402161
7,UAL,UAL,11.360001
8,SPG,SPG,10.736149
9,GPS,GPS,10.526312


## S&P 500 Top 10 Worst Performing Stocks Day

In [27]:
percent_change.reset_index()[['Symbols','Company Name','Day Percent Change']].sort_values('Day Percent Change',ascending=True).reset_index(drop=True).head(10).style.background_gradient(cmap='Reds')

,Symbols,Company Name,Day Percent Change
0,CINF,CINF,-12.458858
1,UPS,UPS,-5.967823
2,MSI,MSI,-5.833987
3,WAT,WAT,-5.589454
4,INCY,INCY,-5.156346
5,CNC,CNC,-5.153914
6,MTD,MTD,-4.798307
7,ATVI,ATVI,-4.771842
8,PPG,PPG,-4.697919
9,NOV,NOV,-4.696356


## S&P 500 Top 10 Best Performing Stocks Week

In [28]:
percent_change.reset_index()[['Symbols','Company Name','Week Percent Change']].sort_values('Week Percent Change',ascending=False).reset_index(drop=True).head(10).style.background_gradient(cmap='Greens')

,Symbols,Company Name,Week Percent Change
0,ADS,ADS,32.332491
1,CPRI,CPRI,31.010458
2,PVH,PVH,28.705814
3,HAL,HAL,27.613947
4,EXPE,EXPE,25.484710
5,LEG,LEG,25.009270
6,DXC,DXC,24.774460
7,COF,COF,24.609070
8,APA,APA,24.504089
9,LNC,LNC,24.253594


## S&P 500 Best Performing Stocks Month

In [29]:
percent_change.reset_index()[['Symbols','Company Name','Month Percent Change']].sort_values('Month Percent Change',ascending=False).reset_index(drop=True).head(10).style.background_gradient(cmap='Greens')

,Symbols,Company Name,Month Percent Change
0,APA,APA,109.169581
1,DVN,DVN,64.069956
2,FANG,FANG,55.383322
3,HOLX,HOLX,47.865301
4,HAL,HAL,46.165890
5,WHR,WHR,40.505574
6,NBL,NBL,40.485066
7,WMB,WMB,38.892882
8,CMG,CMG,38.429675
9,INCY,INCY,38.384573


## S&P 500 Worst Performing Stocks Month

In [30]:
percent_change.reset_index()[['Symbols','Company Name','Month Percent Change']].sort_values('Month Percent Change',ascending=True).reset_index(drop=True).head(10).style.background_gradient(cmap='Reds')

,Symbols,Company Name,Month Percent Change
0,LUV,LUV,-27.899943
1,AAL,AAL,-26.495722
2,BA,BA,-25.123455
3,UAL,UAL,-21.863582
4,DAL,DAL,-21.590522
5,TDG,TDG,-14.186937
6,GE,GE,-11.548552
7,CCL,CCL,-10.687023
8,NCLH,NCLH,-10.149752
9,IVZ,IVZ,-9.498393


## S&P 500 Best Performing Stocks 3 Month

In [31]:
percent_change.reset_index()[['Symbols','Company Name','3 Month Percent Change']].sort_values('3 Month Percent Change',ascending=False).reset_index(drop=True).head(10).style.background_gradient(cmap='Greens')

,Symbols,Company Name,3 Month Percent Change
0,REGN,REGN,53.629972
1,NEM,NEM,41.866652
2,COG,COG,39.632909
3,NLOK,NLOK,31.917475
4,INCY,INCY,27.739994
5,AMZN,AMZN,24.866050
6,GILD,GILD,23.372617
7,MKTX,MKTX,22.879786
8,SBAC,SBAC,21.472869
9,CLX,CLX,20.443042


## S&P 500 Worst Performing Stocks 3 Month

In [32]:
percent_change.reset_index()[['Symbols','Company Name','3 Month Percent Change']].sort_values('3 Month Percent Change',ascending=True).reset_index(drop=True).head(10).style.background_gradient(cmap='Reds')

,Symbols,Company Name,3 Month Percent Change
0,NCLH,NCLH,-76.200000
1,CCL,CCL,-68.540287
2,RCL,RCL,-65.122619
3,UAL,UAL,-63.867620
4,APA,APA,-62.751306
5,OXY,OXY,-62.063842
6,NBL,NBL,-60.890018
7,BA,BA,-58.276095
8,AAL,AAL,-57.986574
9,OKE,OKE,-57.974782


## S&P 500 Top 10 Best Performing Stocks Year

In [33]:
percent_change.reset_index()[['Symbols','Company Name','Year Percent Change']].sort_values('Year Percent Change',ascending=False).reset_index(drop=True).head(10).style.background_gradient(cmap='Greens')

,Symbols,Company Name,Year Percent Change
0,NEM,NEM,102.041896
1,AMD,AMD,99.103300
2,ETN,ETN,70.789549
3,MKTX,MKTX,64.743468
4,NVDA,NVDA,64.170660
5,LDOS,LDOS,56.181540
6,RMD,RMD,53.927147
7,REGN,REGN,53.817721
8,EQIX,EQIX,53.683898
9,SBAC,SBAC,52.538121


## S&P 500 Top 10 Worst Performing Stocks Year

In [34]:
percent_change.reset_index()[['Symbols','Company Name','Year Percent Change']].sort_values('Year Percent Change',ascending=True).reset_index(drop=True).head(10).style.background_gradient(cmap='Reds')

,Symbols,Company Name,Year Percent Change
0,NCLH,NCLH,-77.143356
1,OXY,OXY,-73.092018
2,CCL,CCL,-72.244593
3,DXC,DXC,-71.318085
4,MRO,MRO,-70.820349
5,KSS,KSS,-70.404900
6,NBL,NBL,-69.214536
7,ADS,ADS,-68.987732
8,UAL,UAL,-68.342052
9,FTI,FTI,-66.894708


## Insert S&P 500 Data into Database

In [35]:
conx_string = "driver={SQL Server}; server=DESKTOP-UABFC0C\SQLEXPRESS; database=stocks; trusted_connection=YES;"
conx = pyodbc.connect(conx_string)
cursor = conx.cursor();
sp500_data=df.stack().reset_index().values.tolist()

with pyodbc.connect(conx_string) as conx:
    cursor = conx.cursor()
    cursor.execute("""TRUNCATE table sp500 """)
    cursor.executemany("""INSERT INTO sp500 ("Date", Symbols, "Adj_Close", "Close", "High", "Low", "Open", "Volume") VALUES(?,?,?,?,?,?,?,?)""", sp500_data)

## Get Nasdaq 100 Historical data

In [36]:
nas=web.DataReader(list(nas100.index),'yahoo',start,end)
nas

Attributes  Adj Close                                                 \
Symbols          ATVI        ADBE        AMD        ALXN        ALGN   
Date                                                                   
2019-03-28  45.321922  263.529999  25.059999  133.259995  273.910004   
2019-03-29  45.232513  266.489990  25.520000  135.179993  284.329987   
2019-04-01  46.812122  272.170013  26.360001  135.320007  285.149994   
2019-04-02  47.149902  271.350006  26.750000  138.919998  292.029999   
2019-04-03  47.338661  271.500000  29.020000  139.990005  289.269989   
...               ...         ...        ...         ...         ...   
2020-04-22  66.980003  336.769989  55.919998  103.910004  190.029999   
2020-04-23  65.879997  335.369995  55.900002  104.139999  188.570007   
2020-04-24  66.650002  344.100006  56.180000  108.120003  191.970001   
2020-04-27  67.059998  348.500000  56.490002  110.860001  196.669998   
2020-04-28  63.860001  333.450012  55.509998  106.919998  203.509995   

Attributes                                                                ...  \
Symbols           GOOGL         GOOG         AMZN        AAL        AMGN  ...   
Date                                                                      ...   
2019-03-28  1172.270020  1168.489990  1773.420044  30.491201  182.226471  ...   
2019-03-29  1176.890015  1173.310059  1780.750000  31.339823  184.489136  ...   
2019-04-01  1198.979980  1194.430054  1814.189941  31.922018  186.227402  ...   
2019-04-02  1205.540039  1200.489990  1813.979980  32.553551  186.868301  ...   
2019-04-03  1210.810059  1205.920044  1820.699951  33.264023  187.344147  ...   
...                 ...          ...          ...        ...         ...  ...   
2020-04-22  1258.410034  1263.209961  2363.489990  10.270000  229.289993  ...   
2020-04-23  1271.170044  1276.310059  2399.449951  10.250000  232.490005  ...   
2020-04-24  1276.599976  1279.310059  2410.219971  10.310000  236.279999  ...   
2020-04-27  1270.859985  1275.880005  2376.000000  10.020000  242.429993  ...   
2020-04-28  1232.589966  1233.670044  2314.080078  11.260000  237.669998  ...   

Attributes      Volume                                                         \
Symbols            UAL      VRSN       VRSK       VRTX         WBA       WDAY   
Date                                                                            
2019-03-28   1563100.0  370900.0  1477700.0  1615200.0   5768300.0   838100.0   
2019-03-29   2449900.0  732600.0   979900.0  1274700.0   4993100.0  1081400.0   
2019-04-01   2723200.0  630500.0   775700.0  1030000.0   6328600.0  1223400.0   
2019-04-02   3795100.0  449500.0   636900.0   814400.0  36877800.0   720300.0   
2019-04-03   4047800.0  437500.0  1238700.0   829400.0  17901900.0  1271800.0   
...                ...       ...        ...        ...         ...        ...   
2020-04-22  90730600.0  604500.0   710200.0  1546000.0   4676800.0  1226000.0   
2020-04-23  35812500.0  946700.0  1082800.0  2770400.0   4232100.0  1283200.0   
2020-04-24  34850700.0  911400.0   975900.0  1805800.0   4102400.0  2065600.0   
2020-04-27  35980000.0  593200.0   797700.0  2181600.0   6705500.0  1266700.0   
2020-04-28  59688800.0  898500.0   672600.0  2332300.0   5706100.0  1323400.0   

Attributes                                              
Symbols           WDC       WLTW        XEL       XLNX  
Date                                                    
2019-03-28  5710600.0   607100.0  3436600.0  2513600.0  
2019-03-29  8326300.0   745400.0  5170300.0  4083000.0  
2019-04-01  6579500.0   802200.0  3267000.0  3872700.0  
2019-04-02  6509000.0   696100.0  3684300.0  2668600.0  
2019-04-03  8240200.0   717900.0  3936700.0  3458700.0  
...               ...        ...        ...        ...  
2020-04-22  4630100.0   612900.0  2515600.0  5202200.0  
2020-04-23  5167500.0  1938800.0  3083700.0  7127300.0  
2020-04-24  4208000.0  1441400.0  2868800.0  3273800.0  
2020-04-27  4864800.0   873600.0  275

In [37]:
nas

Attributes  Adj Close                                                 \
Symbols          ATVI        ADBE        AMD        ALXN        ALGN   
Date                                                                   
2019-03-28  45.321922  263.529999  25.059999  133.259995  273.910004   
2019-03-29  45.232513  266.489990  25.520000  135.179993  284.329987   
2019-04-01  46.812122  272.170013  26.360001  135.320007  285.149994   
2019-04-02  47.149902  271.350006  26.750000  138.919998  292.029999   
2019-04-03  47.338661  271.500000  29.020000  139.990005  289.269989   
...               ...         ...        ...         ...         ...   
2020-04-22  66.980003  336.769989  55.919998  103.910004  190.029999   
2020-04-23  65.879997  335.369995  55.900002  104.139999  188.570007   
2020-04-24  66.650002  344.100006  56.180000  108.120003  191.970001   
2020-04-27  67.059998  348.500000  56.490002  110.860001  196.669998   
2020-04-28  63.860001  333.450012  55.509998  106.919998  203.509995   

Attributes                                                                ...  \
Symbols           GOOGL         GOOG         AMZN        AAL        AMGN  ...   
Date                                                                      ...   
2019-03-28  1172.270020  1168.489990  1773.420044  30.491201  182.226471  ...   
2019-03-29  1176.890015  1173.310059  1780.750000  31.339823  184.489136  ...   
2019-04-01  1198.979980  1194.430054  1814.189941  31.922018  186.227402  ...   
2019-04-02  1205.540039  1200.489990  1813.979980  32.553551  186.868301  ...   
2019-04-03  1210.810059  1205.920044  1820.699951  33.264023  187.344147  ...   
...                 ...          ...          ...        ...         ...  ...   
2020-04-22  1258.410034  1263.209961  2363.489990  10.270000  229.289993  ...   
2020-04-23  1271.170044  1276.310059  2399.449951  10.250000  232.490005  ...   
2020-04-24  1276.599976  1279.310059  2410.219971  10.310000  236.279999  ...   
2020-04-27  1270.859985  1275.880005  2376.000000  10.020000  242.429993  ...   
2020-04-28  1232.589966  1233.670044  2314.080078  11.260000  237.669998  ...   

Attributes      Volume                                                         \
Symbols            UAL      VRSN       VRSK       VRTX         WBA       WDAY   
Date                                                                            
2019-03-28   1563100.0  370900.0  1477700.0  1615200.0   5768300.0   838100.0   
2019-03-29   2449900.0  732600.0   979900.0  1274700.0   4993100.0  1081400.0   
2019-04-01   2723200.0  630500.0   775700.0  1030000.0   6328600.0  1223400.0   
2019-04-02   3795100.0  449500.0   636900.0   814400.0  36877800.0   720300.0   
2019-04-03   4047800.0  437500.0  1238700.0   829400.0  17901900.0  1271800.0   
...                ...       ...        ...        ...         ...        ...   
2020-04-22  90730600.0  604500.0   710200.0  1546000.0   4676800.0  1226000.0   
2020-04-23  35812500.0  946700.0  1082800.0  2770400.0   4232100.0  1283200.0   
2020-04-24  34850700.0  911400.0   975900.0  1805800.0   4102400.0  2065600.0   
2020-04-27  35980000.0  593200.0   797700.0  2181600.0   6705500.0  1266700.0   
2020-04-28  59688800.0  898500.0   672600.0  2332300.0   5706100.0  1323400.0   

Attributes                                              
Symbols           WDC       WLTW        XEL       XLNX  
Date                                                    
2019-03-28  5710600.0   607100.0  3436600.0  2513600.0  
2019-03-29  8326300.0   745400.0  5170300.0  4083000.0  
2019-04-01  6579500.0   802200.0  3267000.0  3872700.0  
2019-04-02  6509000.0   696100.0  3684300.0  2668600.0  
2019-04-03  8240200.0   717900.0  3936700.0  3458700.0  
...               ...        ...        ...        ...  
2020-04-22  4630100.0   612900.0  2515600.0  5202200.0  
2020-04-23  5167500.0  1938800.0  3083700.0  7127300.0  
2020-04-24  4208000.0  1441400.0  2868800.0  3273800.0  
2020-04-27  4864800.0   873600.0  275

In [38]:
percent_change_prev_day=pd.Series(((nas['Adj Close'].loc[latest_day]-nas['Adj Close'].loc[previous_day])/nas['Adj Close'].loc[previous_day])*100,name='Day Percent Change')
percent_change_abs_prev_day=pd.Series(abs((nas['Adj Close'].iloc[-1,:]-nas['Adj Close'].loc[previous_day])/nas['Adj Close'].loc[previous_day])*100,name='Day Absolute Percent Change')

percent_change_prev_week=pd.Series(((nas['Adj Close'].iloc[-1,:]-nas['Adj Close'].iloc[-5,:])/nas['Adj Close'].iloc[-5,:])*100,name='Week Percent Change')
percent_change_abs_prev_week=pd.Series(abs((df['Adj Close'].iloc[-1,:]-df['Adj Close'].iloc[-5,:])/nas['Adj Close'].iloc[-5,:])*100,name='Week Absolute Percent Change')

percent_change_prev_month=pd.Series(((nas['Adj Close'].loc[latest_day]-nas['Adj Close'].loc[previous_month])/nas['Adj Close'].loc[previous_month])*100,name='Month Percent Change')
percent_change_abs_prev_month=pd.Series(abs((nas['Adj Close'].iloc[-1,:]-nas['Adj Close'].loc[previous_month])/nas['Adj Close'].loc[previous_month])*100,name='Month Absolute Percent Change')

percent_change_prev_3_month=pd.Series(((nas['Adj Close'].loc[latest_day]-nas['Adj Close'].loc[previous_3_month])/nas['Adj Close'].loc[previous_3_month])*100,name='3 Month Percent Change')
percent_change_abs_prev_3_month=pd.Series(abs((nas['Adj Close'].iloc[-1,:]-nas['Adj Close'].loc[previous_3_month])/nas['Adj Close'].loc[previous_3_month])*100,name='3 Month Absolute Percent Change')

percent_change_prev_year=pd.Series(((nas['Adj Close'].loc[latest_day]-nas['Adj Close'].loc[previous_year])/nas['Adj Close'].loc[previous_year])*100,name='Year Percent Change')
percent_change_abs_prev_year=pd.Series(abs((nas['Adj Close'].iloc[-1,:]-nas['Adj Close'].loc[previous_year])/nas['Adj Close'].loc[previous_year])*100,name='Year Absolute Percent Change')

percent_change=pd.concat([percent_change_prev_day,percent_change_abs_prev_day,percent_change_prev_week,
                          percent_change_abs_prev_week,percent_change_prev_month,percent_change_abs_prev_month,
                          percent_change_prev_3_month,percent_change_abs_prev_3_month,
                          percent_change_prev_year,percent_change_abs_prev_year],axis=1)


percent_change=percent_change.join(nas100)

percent_change=percent_change[percent_change.columns[0:-1].insert(0,percent_change.columns[-1])].reset_index()
percent_change.rename(columns={'index':'Symbols'},inplace=True)

## Nasdaq 100 Top 10 Best Performing Stocks Day

In [40]:
percent_change.reset_index()[['Symbols','Company Name','Day Percent Change']].sort_values('Day Percent Change',ascending=False).reset_index(drop=True).head(10).style.background_gradient(cmap='Greens')

,Symbols,Company Name,Day Percent Change
0,AAL,American Airlines Group,12.375247
1,UAL,United Airlines Holdings,11.360001
2,CPRT,Copart,6.020773
3,ULTA,Ulta Beauty,5.089142
4,EXPE,Expedia Group,3.890086
5,WDC,Western Digital,3.736367
6,FOX,Fox Corporation (Class B),3.690885
7,ALGN,"Align Technology, Inc.",3.477905
8,MCHP,Microchip Technology,3.116514
9,FOXA,Fox Corporation (Class A),2.841986


## Nasdaq 100 Top 10 Worst Performing Stocks Day

In [41]:
percent_change.reset_index()[['Symbols','Company Name','Day Percent Change']].sort_values('Day Percent Change',ascending=True).reset_index(drop=True).head(10).style.background_gradient(cmap='Reds')

,Symbols,Company Name,Day Percent Change
0,INCY,Incyte Corporation,-5.156346
1,ATVI,Activision Blizzard,-4.771842
2,JD,JD.com,-4.387889
3,ADBE,Adobe Inc.,-4.318504
4,TTWO,"Take-Two Interactive, Inc.",-4.303977
5,VRTX,Vertex Pharmaceuticals,-4.301522
6,CTXS,Citrix Systems,-4.264981
7,IDXX,IDEXX Laboratories,-4.205186
8,NFLX,Netflix,-4.164891
9,EA,Electronic Arts,-4.025167


## Nasdaq 100 Top 10 Best Performing Stocks Month

In [42]:
percent_change.reset_index()[['Symbols','Company Name','Month Percent Change']].sort_values('Month Percent Change',ascending=False).reset_index(drop=True).head(10).style.background_gradient(cmap='Greens')

,Symbols,Company Name,Month Percent Change
0,TSLA,"Tesla, Inc.",49.529516
1,INCY,Incyte Corporation,41.632830
2,SGEN,Seattle Genetics,31.207101
3,EBAY,eBay Inc.,29.489733
4,MCHP,Microchip Technology,28.079182
5,ALXN,Alexion Pharmaceuticals,27.270558
6,MELI,MercadoLibre,26.962643
7,CDNS,Cadence Design Systems,25.995509
8,KHC,Kraft Heinz,25.536306
9,ULTA,Ulta Beauty,25.353968


## Nasdaq 100 Top 10 Worst Performing Stocks Month

In [43]:
percent_change.reset_index()[['Symbols','Company Name','Month Percent Change']].sort_values('Month Percent Change',ascending=True).reset_index(drop=True).head(10).style.background_gradient(cmap='Reds')

,Symbols,Company Name,Month Percent Change
0,AAL,American Airlines Group,-19.800568
1,UAL,United Airlines Holdings,-15.225335
2,BIDU,"Baidu.com, Inc.",-0.512138
3,CTXS,Citrix Systems,0.821201
4,WBA,"Walgreen Boots Alliance, Inc.",1.477276
5,ISRG,Intuitive Surgical Inc.,2.659001
6,BIIB,"Biogen, Inc",2.854255
7,MU,"Micron Technology, Inc.",4.208836
8,SPLK,Splunk,4.272198
9,TCOM,Trip.com Group,4.305201


## Nasdaq 100 Top 10 Best Performing Stocks 3 Month

In [44]:
percent_change.reset_index()[['Symbols','Company Name','3 Month Percent Change']].sort_values('3 Month Percent Change',ascending=False).reset_index(drop=True).head(10).style.background_gradient(cmap='Greens')

,Symbols,Company Name,3 Month Percent Change
0,REGN,Regeneron Pharmaceuticals,53.629972
1,TSLA,"Tesla, Inc.",35.671188
2,INCY,Incyte Corporation,27.739994
3,SGEN,Seattle Genetics,26.954746
4,AMZN,Amazon.com,24.866050
5,GILD,"Gilead Sciences, Inc.",23.372617
6,NVDA,NVIDIA Corporation,17.568354
7,NFLX,Netflix,15.869964
8,CTXS,Citrix Systems,12.815720
9,JD,JD.com,12.002061


## Nasdaq 100 Top 10 Worst Performing Stocks 3 Month

In [45]:
percent_change.reset_index()[['Symbols','Company Name','3 Month Percent Change']].sort_values('3 Month Percent Change',ascending=True).reset_index(drop=True).head(10).style.background_gradient(cmap='Reds')

,Symbols,Company Name,3 Month Percent Change
0,UAL,United Airlines Holdings,-63.867620
1,AAL,American Airlines Group,-57.986574
2,MAR,"Marriott International, Inc.",-37.861577
3,EXPE,Expedia Group,-33.913561
4,WDC,Western Digital,-33.802752
5,FOXA,Fox Corporation (Class A),-26.397946
6,FOX,Fox Corporation (Class B),-25.814991
7,TCOM,Trip.com Group,-25.735967
8,NXPI,NXP Semiconductors N.V.,-25.578367
9,BKNG,Booking Holdings,-24.935723


## Nasdaq 100 Top 10 Best Performing Stocks Year

In [46]:
percent_change.reset_index()[['Symbols','Company Name','Year Percent Change']].sort_values('Year Percent Change',ascending=False).reset_index(drop=True).head(10).style.background_gradient(cmap='Greens')

,Symbols,Company Name,Year Percent Change
0,TSLA,"Tesla, Inc.",227.090243
1,SGEN,Seattle Genetics,108.994167
2,AMD,Advanced Micro Devices,99.103300
3,NVDA,NVIDIA Corporation,64.170660
4,REGN,Regeneron Pharmaceuticals,53.817721
5,JD,JD.com,49.605229
6,VRTX,Vertex Pharmaceuticals,48.660678
7,ASML,ASML Holding,41.991494
8,CTXS,Citrix Systems,41.762994
9,AAPL,Apple Inc.,38.143087
